In [13]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad

import scTRS.data_loader as dl

In [14]:
df_hom = pd.read_csv('/n/holystore01/LABS/price_lab/Users/mjzhang/scTRS_data/gene_annotation/'
                     'mouse_human_homologs.txt', sep='\t')
dic_map = {x:y for x,y in zip(df_hom['HUMAN_GENE_SYM'], df_hom['MOUSE_GENE_SYM'])}
dic_map_m2h = {x:y for x,y in zip(df_hom['MOUSE_GENE_SYM'], df_hom['HUMAN_GENE_SYM'])}

In [15]:
# Richtor 2021 
dic_gs = {}
adata = dl.load_richter_raw_data(opt='raw', flag_size_factor=True, flag_log1p=True)
adata = adata[adata.obs['inferred_cell_label']=='Hepatocytes'].copy()
adata = adata[:,[x for x in adata.var_names if x.startswith('ERCC') is False]]

sc.tl.rank_genes_groups(adata, groupby='Ploidy', method='wilcoxon')

temp_df = sc.get.rank_genes_groups_df(adata, group='4n')
ind_select = (temp_df['logfoldchanges']>0.5) & (temp_df['pvals_adj']<0.01)
print('4n', ind_select.sum())
dic_gs['hep4n_richter'] = [dic_map_m2h[x] for x in temp_df.loc[ind_select, 'names'][:200] 
                           if x in dic_map_m2h]
ind_select = (temp_df['logfoldchanges']<-0.1) & (temp_df['pvals_adj']<0.01)
print('2n', ind_select.sum())
dic_gs['hep2n_richter'] = [dic_map_m2h[x] for x in temp_df['names'][-200:] 
                           if x in dic_map_m2h]

Trying to set attribute `.uns` of view, copying.


4n 830
2n 5


In [16]:
# Table S1 in Miettinen CurrBio 2014

temp_df = pd.read_excel('/n/holystore01/LABS/price_lab/Users/mjzhang/scTRS_data/single_cell_data/'
                        'miettinen_CurrBio_2014/Miettinen_CurrBio_2014_supp_tab.xlsx', skiprows=4)

# ind_select = (temp_df['Pearson correlation with nuclear radius ( r )']>0.8) & \
#                 (temp_df['correlation adj.P.val']<0.01)
# dic_gs['size_pos_meittinen'] = [dic_map_m2h[x] for x in temp_df.loc[ind_select, 'Gene_Name']
#                                 if x in dic_map_m2h] 

# ind_select = (temp_df['Pearson correlation with nuclear radius ( r )']<-0.8) & \
#                 (temp_df['correlation adj.P.val']<0.01)
# dic_gs['size_neg_meittinen'] = [dic_map_m2h[x] for x in temp_df.loc[ind_select, 'Gene_Name']
#                                 if x in dic_map_m2h] 

# # 
# ind_select = (temp_df['[Cdk1Flox/Flox_Pre-PH] vs [Cdk1Liv-/-_Pre-PH] logFC']>0.5) & \
#                 (temp_df['[Cdk1Flox/Flox_Pre-PH] vs [Cdk1Liv-/-_Pre-PH] adj.P.val']<0.01)
# dic_gs['ctrl_vs_case_prePH'] = [dic_map_m2h[x] for x in temp_df.loc[ind_select, 'Gene_Name']
#                                 if x in dic_map_m2h]

# ind_select = (temp_df['[Cdk1Flox/Flox_Pre-PH] vs [Cdk1Liv-/-_Pre-PH] logFC']<-0.5) & \
#                 (temp_df['[Cdk1Flox/Flox_Pre-PH] vs [Cdk1Liv-/-_Pre-PH] adj.P.val']<0.01)
# dic_gs['case_vs_ctrl_prePH'] = [dic_map_m2h[x] for x in temp_df.loc[ind_select, 'Gene_Name']
#                                 if x in dic_map_m2h]

# 
ind_select = (temp_df['[Cdk1Flox/Flox_96hPH] vs [Cdk1Liv-/-_96hPH] logFC']>0.5) & \
                (temp_df['[Cdk1Flox/Flox_96hPH] vs [Cdk1Liv-/-_96hPH] adj.P.val']<0.01)
dic_gs['ctrl_vs_case_96hPH'] = [dic_map_m2h[x] for x in temp_df.loc[ind_select, 'Gene_Name']
                                if x in dic_map_m2h]

ind_select = (temp_df['[Cdk1Flox/Flox_96hPH] vs [Cdk1Liv-/-_96hPH] logFC']<-0.5) & \
                (temp_df['[Cdk1Flox/Flox_96hPH] vs [Cdk1Liv-/-_96hPH] adj.P.val']<0.01)
dic_gs['case_vs_ctrl_96hPH'] = [dic_map_m2h[x] for x in temp_df.loc[ind_select, 'Gene_Name'] 
                               if x in dic_map_m2h] 



# 
ind_select = (temp_df['[Cdk1Liv-/-_Pre-PH] vs [Cdk1Liv-/-_96hPH] logFC']>0.5) & \
                (temp_df['[Cdk1Liv-/-_Pre-PH] vs [Cdk1Liv-/-_96hPH] adj.P.val']<0.01)
dic_gs['case_prePH_vs_96hPH'] = [dic_map_m2h[x] for x in temp_df.loc[ind_select, 'Gene_Name'] 
                                 if x in dic_map_m2h]

ind_select = (temp_df['[Cdk1Liv-/-_Pre-PH] vs [Cdk1Liv-/-_96hPH] logFC']<-0.5) & \
                (temp_df['[Cdk1Liv-/-_Pre-PH] vs [Cdk1Liv-/-_96hPH] adj.P.val']<0.01)
dic_gs['case_96hPH_vs_prePH'] = [dic_map_m2h[x] for x in temp_df.loc[ind_select, 'Gene_Name'] 
                                 if x in dic_map_m2h]

dic_gs['large_vs_small'] = temp_df.sort_values('log2FoldChange(Largest vs smallest)')['Gene_Name'][:500]
dic_gs['large_vs_small'] = [dic_map_m2h[x] for x in dic_gs['large_vs_small'] 
                            if x in dic_map_m2h] 

In [17]:
for term in dic_gs:
    print(term, len(dic_gs[term]), 
          'overlap with hep4n_richtor', len(set(dic_gs[term]) & set(dic_gs['hep4n_richter'])),
          'overlap with hep2n_richtor', len(set(dic_gs[term]) & set(dic_gs['hep2n_richter'])))

hep4n_richter 144 overlap with hep4n_richtor 144 overlap with hep2n_richtor 0
hep2n_richter 134 overlap with hep4n_richtor 0 overlap with hep2n_richtor 134
ctrl_vs_case_96hPH 509 overlap with hep4n_richtor 1 overlap with hep2n_richtor 12
case_vs_ctrl_96hPH 399 overlap with hep4n_richtor 30 overlap with hep2n_richtor 7
case_prePH_vs_96hPH 784 overlap with hep4n_richtor 3 overlap with hep2n_richtor 20
case_96hPH_vs_prePH 472 overlap with hep4n_richtor 39 overlap with hep2n_richtor 5
large_vs_small 261 overlap with hep4n_richtor 16 overlap with hep2n_richtor 1


In [18]:
df_gs = pd.DataFrame(columns=['TRAIT', 'GENESET'])
for term in dic_gs:
    df_gs.loc[term] = [term, ','.join(dic_gs[term])]
df_gs.to_csv('/n/holystore01/LABS/price_lab/Users/mjzhang/scTRS_data/gs_file/ploidy.gs',
             sep='\t', index=False)

### Old code 

In [ ]:
import matplotlib.pyplot as plt
adata = dl.load_richter_raw_data(opt='raw', flag_size_factor=True, flag_log1p=True)
adata = adata[adata.obs['inferred_cell_label']=='Hepatocytes'].copy()
adata = adata[:,[x for x in adata.var_names if x.startswith('ERCC') is False]]

sc.tl.rank_genes_groups(adata, groupby='Ploidy', method='wilcoxon')
temp_df = sc.get.rank_genes_groups_df(adata, group='4n')
temp_df['mean'] = adata[:,temp_df['names'].values].X.mean(axis=0).T
temp_df['rank'] = np.arange(temp_df.shape[0])
temp_df['rank_inv'] = np.arange(temp_df.shape[0])[::-1]

plt.figure()
ind_select = temp_df['pvals_adj']<0.05
plt.scatter(temp_df.loc[ind_select, 'mean'], temp_df.loc[ind_select, 'logfoldchanges'], s=4)
ind_select = [x.startswith('ERCC') for x in temp_df['names']]
plt.scatter(temp_df.loc[ind_select, 'mean'], temp_df.loc[ind_select, 'logfoldchanges'], s=4)
plt.plot([0,5], [0,0])
# plt.xlim([0,5])
plt.show()

temp_list = ['Alb', 'Mup20', 'Hal', 'Sds', 'Plbd2',
             'Mcmdc2', 'St5', '30204K15Rik', 'Cryl1', 'Acaca',
             'Malat1', 'Hsd17b13', 'Gls2', 'Lars2', 'mt-Rnr2', 
             'Dab1', 'Gm35188', 'Cyp2f2', 'Tcf7l1', 'Gm14326',
             'Dhdds', 'Zbtb20', 'Etnppl', 'Fhit', 'Gm26885']
display(temp_df.loc[temp_df['names'].isin(temp_list)])

temp_list = ['Cyp2e1', 'Akr1c6', 'Slco1b2', 'Tomm40l', 'Sult2a8', 
             'Nr1i3', 'Cyp27a1', 'Hamp', 'Cd1d1', 'Hsd3b7',
             'Id3', 'Dlec1', 'Rnase4', 'Acaa1b', 'Rgn', 
             'Ang', 'Mup17', 'Gm44662', 'Lifr', 'Cyp2c50', 
             'Mup15', 'Aox1', 'Gm49388', 'Gm36041', 'Csad']
display(temp_df.loc[temp_df['names'].isin(temp_list)])


In [ ]:

# separate the matrix into the 2 dilutions and calculate the ERCC size factor separately 
ERCC = [x for x in adata.var_names if x.startswith('ERCC')]
adata.obs["ERCC_size_factor"] = adata[:,ERCC].X.sum(axis=1)
ind_select = adata.obs["ERCC.dilution"] == "1 in 100.000"
adata.obs.loc[ind_select, "ERCC_size_factor"] /= np.mean(adata[ind_select, ERCC].X.sum(axis=1))
ind_select = adata.obs["ERCC.dilution"] == "1 in 300.000"
adata.obs.loc[ind_select, "ERCC_size_factor"] /= np.mean(adata[ind_select, ERCC].X.sum(axis=1))
adata.X = (adata.X.T / adata.obs["ERCC_size_factor"].values).T*10000
sc.pp.log1p(adata)

In [ ]:
# Richtor 2021 
adata = dl.load_richter_raw_data(opt='raw', flag_size_factor=False, flag_log1p=False)

ERCC = [x for x in adata.var_names if x.startswith('ERCC')]
# separate the matrix into the 2 dilutions and calculate the ERCC size factor separately 
adataHD = adata[adata.obs["ERCC.dilution"] == "1 in 300.000",:].copy()
adataLD = adata[adata.obs["ERCC.dilution"] == "1 in 100.000",:].copy()
adataHD.obs["ERCC_size_factor"] = np.sum(adataHD[:,ERCC].X, axis=1)/np.mean(np.sum(adataHD[:,ERCC].X, axis=1))
adataLD.obs["ERCC_size_factor"] = np.sum(adataLD[:,ERCC].X, axis=1)/np.mean(np.sum(adataLD[:,ERCC].X, axis=1))

adata = ad.AnnData.concatenate(adataLD, adataHD)
new_names = adataLD.obs_names.append(adataHD.obs_names)
adata.obs_names= new_names
adata = adata[:,[x for x in adata.var_names if x.startswith('ERCC') is False]]

# divide each column by transcript length (per 1kb)
adata.X = adata.X/(adata.var["length"].values/1000) 
adata.X[adata.X>1e8] = 0

adata.obs['n_counts_TPM'] = adata.X.sum(axis=1)

# sum the gene length-corrected counts per cell and divide by 10,000 times the ERCC size factor
adata.obs['RPK_factor'] = adata.obs['n_counts_TPM']/(adata.obs['ERCC_size_factor']*10000)

# divide each row (i.e. nucleus) by the size factor
adata.X = (adata.X.T / adata.obs["RPK_factor"].values).T
adata.obs['n_counts_TPM_norm'] = adata.X.sum(axis=1)

sc.pp.log1p(adata)
sc.pp.combat(adata, key="exp.")